In [ ]:
#!pip install litellm
#!pip install python-dotenv

from dotenv import load_dotenv
import os

load_dotenv()  # Loads variables from .env into environment
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file.")
os.environ['OPENAI_API_KEY'] = api_key # <---- Reference your OpenAI API key here

# There are python files as examples in the /SamplePy directory.
# There are sample data files in the /SampleData directory.

  Using cached litellm-1.73.6-py3-none-any.whl.metadata (39 kB)
  Using cached aiohttp-3.12.13-cp39-cp39-macosx_11_0_arm64.whl.metadata (7.6 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached openai-1.93.0-py3-none-any.whl.metadata (29 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached tiktoken-0.9.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached MarkupSafe-3.0.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (4.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using ca

In [1]:
import json
import os
from typing import List

from litellm import completion

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

#def list_files_recursive(path=".") -> list:
#    """List all files in the directory and subdirectories with their paths."""
#    file_list = []
#    for root, dirs, files in os.walk(path):
#        for file in files:
#            file_list.append(os.path.join(root, file))
#    return file_list

def list_directory_content(path=".") -> dict:
    """List everything in current directory."""
    files = []
    dirs = []
    for entry in os.listdir(path):
        if os.path.isfile(os.path.join(path, entry)):
            files.append(entry)
        elif os.path.isdir(os.path.join(path, entry)):
            dirs.append(entry)
    return {"files": files, "directories": dirs}

def summarize_directory(path=".") -> dict:
    """Summarize what each file in the directory and subdirectories is about, including file paths."""
    summaries = {}
    for root, dirs, files in os.walk(path):
        for file in files:
            full_path = os.path.join(root, file)
            try:
                with open(full_path, "r") as f:
                    content = f.read(1000)  # Read first 1000 chars for summary
                # Use LLM to summarize
                summary_response = completion(
                    model="openai/gpt-4o",
                    messages=[
                        {"role": "system", "content": "Summarize the following file content in 1-2 sentences."},
                        {"role": "user", "content": content}
                    ],
                    max_tokens=100
                )
                summary = summary_response.choices[0].message.content.strip()
            except Exception as e:
                summary = f"Error reading/summarizing: {str(e)}"
            summaries[full_path] = summary
    return summaries

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"

def terminate(message: str) -> None:
    """Terminate the agent loop and provide a summary message."""
    print(f"Termination message: {message}")

tool_functions = {
    "list_files": list_files,
    #"list_files_recursive": list_files_recursive,
    "list_directory_content": list_directory_content,
    "summarize_directory": summarize_directory, 
    "read_file": read_file,
    "terminate": terminate
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
        #{
        #"type": "function",
        #"function": {
        #    "name": "list_files_recursive",
        #    "description": "Returns a list of files in the directory and sub directories with their paths.",
        #    "parameters": {"type": "object", "properties": {}, "required": []}
        #}
    #},
        {
        "type": "function",
        "function": {
            "name": "list_directory_content",
            "description": "Returns the content of the current directory.",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
        {
        "type": "function",
        "function": {
            "name": "summarize_directory",
            "description": "Returns a summary of the content in specified directory.",
            "parameters": {
                 "type": "object",
                 "properties": {
                    "path": {"type": "string", "description": "The directory path to summarize."}
                    },
                "required": ["path"]
             }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},
                },
                "required": ["message"]
            }
        }
    }
]

agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.
"""
}]

# Initialize agent parameters
iterations = 0
max_iterations = 10

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

# The Agent Loop
while iterations < max_iterations:

    messages = agent_rules + memory
    iterations += 1

    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        tools=tools,
        max_tokens=1024
    )

    if response.choices[0].message.tool_calls:
        tool = response.choices[0].message.tool_calls[0]
        tool_name = tool.function.name
        tool_args = json.loads(tool.function.arguments)

        action = {
            "tool_name": tool_name,
            "args": tool_args
        }

        if tool_name == "terminate":
            print(f"Termination message: {tool_args['message']}")
            break
        elif tool_name in tool_functions:
            try:
                result = {"result": tool_functions[tool_name](**tool_args)}
            except Exception as e:
                result = {"error":f"Error executing {tool_name}: {str(e)}"}
        else:
            result = {"error": f"Unknown tool: {tool_name}"}

        print(f"Executing: {tool_name} with args {tool_args}")
        print(f"Result: {result}")
        memory.extend([
            {"role": "assistant", "content": json.dumps(action)},
            {"role": "user", "content": json.dumps(result)}
        ])
    else:
        result = response.choices[0].message.content
        print(f"Response: {result}")
        break


Executing: list_files with args {}
Result: {'result': ['.DS_Store', 'Archive', 'SampleData', 'ASimpleAgentFramework.ipynb', 'README.md', '.gitignore', '.env', '.venv', 'SamplePy', '.git', 'AgentLoopWithFunctionCalling.ipynb']}
Executing: list_directory_content with args {}
Result: {'result': {'files': ['.DS_Store', 'ASimpleAgentFramework.ipynb', 'README.md', '.gitignore', '.env', 'AgentLoopWithFunctionCalling.ipynb'], 'directories': ['Archive', 'SampleData', '.venv', 'SamplePy', '.git']}}
Executing: summarize_directory with args {'path': 'SampleData'}
Result: {'result': {'SampleData/.DS_Store': "Error reading/summarizing: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte", 'SampleData/Altertum.csv': 'The file lists various historical conflicts, including the Lagaš-Umma War, Akkad-Nagar and Akkad-Ebla War, Babylon-Mari War, the Battle of Megiddo, and the Battle of Qadeš, detailing their time frames, main aggressors, involved parties, and types of conflicts within